In [2]:
import dask.dataframe as dd
from datetime import datetime
import math
from dask.multiprocessing import get
from dask.distributed import Client, LocalCluster 
print('test')

#cluster = LocalCluster(n_workers = 8, threads_per_worker = 16, processes=False, memory_limit='16GB', silence_logs='error')
client = Client(threads_per_worker=32, memory_limit='16GB', processes=False, silence_logs='error')
#client = Client(cluster)

def function_to_apply_row(x, y):
    return x*y

def function_to_apply_row_v1(x):
    return len(str(x).split(' '))
    

test


In [3]:
%timeit -n1 -r7 df = dd.read_csv('./data/Resources.csv',header=0, sep=',').compute(scheduler='threads')
#7.2 s ± 201 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

7.38 s ± 387 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
df = dd.read_csv('./data/Resources.csv',header=0, sep=',')

In [4]:
%%timeit -n1 -r10 
df.to_csv('./output/output.csv', mode="w+")
#41.6 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

FileExistsError: [Errno 17] File exists: '/Users/tomasp/BlogPosts/output/output.csv'

In [3]:
%%timeit -n1 -r7 
df["cost"]=df["Resource Unit Price"]*df["Resource Quantity"]
#270 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

5.15 ms ± 914 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit -n1 -r7 
df["cost"] = df.map_partitions(lambda df1: df1.apply(lambda x : x["Resource Unit Price"]*x["Resource Quantity"], axis=1))
df.compute(scheduler='threads')


In [0]:
%%timeit -n1 -r7 
df["cost"] = df.map_partitions(lambda df1: df1.apply(lambda x : function_to_apply_row(x["Resource Unit Price"], x["Resource Quantity"]), axis=1))
df.compute(scheduler='threads')

In [0]:
%timeit -n1 -r7 agg_df = df.groupby("Resource Vendor Name").agg({"Resource Quantity" : 'mean'}).compute(scheduler='threads')

%timeit -n1 -r7 agg_df = df.fillna('0').compute(scheduler='threads')

In [0]:
del df

In [0]:
%%timeit -n1 -r7 
df = dd.read_csv('./data/resources_v2.csv',header=0, sep=',').compute(scheduler='threads')

In [1]:
df = dd.read_csv('./data/resources_v2.csv',header=0, sep=',')

NameError: name 'dd' is not defined

In [0]:
%%timeit -n1 -r7 
df.to_csv('./output/output.csv', mode="w+")

In [0]:
%%timeit -n1 -r7 
df["cost"]=df["Resource Unit Price"]*df["Resource Quantity"]
df.compute(scheduler='threads')

In [0]:
%%timeit -n1 -r7 
df["cost"] = df.map_partitions(lambda df1: df1.apply(lambda x : x["Resource Unit Price"]*x["Resource Quantity"], axis=1)).compute(scheduler='threads')


In [0]:
%%timeit -n1 -r7 
df["cost"] = df.map_partitions(lambda df1: df1.apply(lambda x : function_to_apply_row(x["Resource Unit Price"], x["Resource Quantity"]), axis=1)).compute(scheduler='threads')

In [0]:
%timeit -n1 -r7 agg_df = df.groupby("Resource Vendor Name").agg({"Resource Quantity" : 'mean'}).compute(scheduler='threads')

%timeit -n1 -r7 agg_df = df.fillna('0').compute(scheduler='threads')

In [0]:
del df

In [0]:
del df
client.shutdown()